In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import scipy
# import tushare as ts
import wrds
import talib as ta
import datetime, time


In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

retrieve data from csv

In [29]:
# data_query7 = pd.read_csv('data/query7-replace some data in query6.zip',
#                          parse_dates=['datadate'], infer_datetime_format=True)

In [4]:
data_return = pd.read_csv('data/return.zip', parse_dates=['date'], infer_datetime_format=True,
                  dtype={'PERMCO':str, 'CUSIP':str, 'PERMNO':str})
# data = pd.read_csv('query3.csv', parse_dates=['date'], infer_datetime_format=True,)
                   #dtype={'PRC':np.float64, 'SHROUT':np.float64})
data_return = data_return.set_index('date')
data_return.RET = data_return.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
data_return.RETX = data_return.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
# data_return = data_return.assign(MV=data_return.PRC.abs() * data_return.SHROUT,
#                           RET_PLUS=data_return.RET+1,
#                           RETX_PLUS=data_return.RETX+1)
# data_return = data_return.set_index(['date', 'PERMCO'])

In [5]:
def calc_basis(dataframe):
    PRC = dataframe.PRC
    SHROUT = dataframe.SHROUT
    CUSIP = dataframe.CUSIP
    MV = PRC.abs() * SHROUT
    RET_CUM = (dataframe.RET + 1).cumprod()
    RETX_CUM = (dataframe.RETX + 1).cumprod()
    
    return pd.DataFrame(
        {
            'CUSIP':CUSIP,
            'PRC':PRC,
            'SHROUT':SHROUT,
            'MV':MV,
            'RET_CUM':RET_CUM,
            'RETX_CUM':RETX_CUM
        }
    )

In [ ]:
data_ret_cum = data_return.groupby(['PERMCO', pd.Grouper(freq="Y")]).\
apply(lambda x : calc_basis(x))

In [25]:
data_return.head(1)

,,PERMNO,TICKER,COMNAM,CUSIP,PRC,RET,SHROUT,RETX,MV,RET_PLUS,RETX_PLUS
date,PERMCO,,,,,,,,,,,
1973-01-31,2,14403,AAGIY,ANGLO AMERICAN GOLD INVT LTD,03487050,-27.125,0.179348,21952.0,0.179348,595448.0,1.179348,1.179348


In [ ]:
data_return_year = data_return.groupby(level=1, sort=False).resample('Y', level=0).apply(
                    {
                        'PERMNO':'last', 
                        'TICKER':'last', 
                        'COMNAM':'last', 
                        'CUSIP':'last', 
                        'PRC':'last', 
                        'RET':'last', 
                        'SHROUT':'last', 
                        'RETX':'last',
                        'MV':'last', 
                        'RET_PLUS':'cumprod', 
                        'RETX_PLUS':'cumprod'
                    })

In [1]:
# data_query7.head()

calc MV & CUM RETURNX

In [5]:
cum_ret = data.groupby(['PERMCO', pd.Grouper(freq="Y")]).apply(lambda x : x.RETPLUS.cumprod())
cum_ret.index = cum_ret.index.droplevel(1)
cum_ret = cum_ret.reset_index()
cum_ret.columns = ['PERMCO', 'date', 'CUMRETX']
cum_ret = cum_ret.set_index('date')
cum_ret.head()

,PERMCO,date,CUMRETX
0,100,1973-01-31,0.763819
1,100,1973-02-28,0.733668
2,100,1973-03-30,0.713568
3,100,1973-04-30,0.693467
4,100,1973-05-31,0.542714
